In [1]:
# python detect_mask_video.py

# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os

In [2]:
# to capture the image 
def detect_and_predict_mask(frame,faceNet,maskNet):
    # to get the dimension of the frame and to construct the blob
    (h,w)= frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame,1.0,(300,300),
                                 (104.0,177.0,123.0))
    
    #pass the blob through model and detect the face using faceNet
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    #initialize the list of face and the corresponding locatins and the list of predicitons from our face mask model
    
    faces =[]
    locs =[]
    preds =[]
    
    # loop over detections
    for i in range(0,detections.shape[2]):
        # extract the probability
        confidence = detections[0,0,i,2]
        
        #filter out weak detections by ensuring the confidence/probability value
        # (i.e) the confidence value is greater than the min conf value
        
        if confidence>args['confidence']:
            #compute the x and y coordinates of bounding box 
            
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype("int")
            
            #ensure the box falls within the face dim
            (startX,startY) =(max(0,startX),max(0,startY))
            (endX,endY) = (min(w-1,endX),min(h-1,endY))
            face = frame[startY:endY,startX:endX]

            # Debugging statements
            print(f"Detection {i}: confidence={confidence}")
            print(f"Bounding box: startX={startX}, startY={startY}, endX={endX}, endY={endY}")
            print(f"Face shape: {face.shape}")

            if face.size == 0:
                print("Empty face image detected, skipping...")
                continue
            
            #extract the face ROI, convert it to BGR to RGB channel ordering, resize it to 224x224, and preprocess it
            # face = frame[startY:endY,startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face,(224,224))
            face = img_to_array(face)
            face = preprocess_input(face)
            
            # add the face and bounding box to the respective list
            faces.append(face)
            locs.append((startX,startY,endX,endY))
            
    # to give a condition to predict atleast one face was detected
    if len(faces)>0:
# for faster inference we'll make batch predictions on *all*
# faces at the same time rather than one-by-one predictions
# in the above `for` loop
        faces = np.array(faces,dtype ='float32')
        preds = maskNet.predict(faces,batch_size = 32)
        
    return (locs,preds)
        


In [3]:
# construct the argument parser and parse the arguments
# Construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-f", "--face", type=str, default="face_detector", help="path to face detector model directory")
ap.add_argument("-m", "--model", type=str, default="mask_detector.h5", help="path to trained face mask detector model")
ap.add_argument("-c", "--confidence", type=float, default=0.5, help="minimum probability to filter weak detections")
args = vars(ap.parse_args())

# Construct the absolute paths to the model files
project_root = os.path.abspath(os.path.dirname(r"C:\HOPE\Deep Learning\Face Mask Dectection\face_detector"))
face_detector_directory = os.path.join(project_root, 'face_detector')
prototxtPath = os.path.join(face_detector_directory, "deploy.prototxt")
weightsPath = os.path.join(face_detector_directory, "res10_300x300_ssd_iter_140000.caffemodel")

print(f"Prototxt path: {prototxtPath}")
print(f"Weights path: {weightsPath}")

if not os.path.exists(prototxtPath):
    print(f"Error: {prototxtPath} not found.")
if not os.path.exists(weightsPath):
    print(f"Error: {weightsPath} not found.")

faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)



Prototxt path: C:\HOPE\Deep Learning\Face Mask Dectection\face_detector\deploy.prototxt
Weights path: C:\HOPE\Deep Learning\Face Mask Dectection\face_detector\res10_300x300_ssd_iter_140000.caffemodel


In [4]:
# Load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
maskNet = load_model(args["model"])


[INFO] loading face mask detector model...


In [5]:
# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)



[INFO] starting video stream...


In [7]:
# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # determine the class label and color we'll use to draw
        # the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # include the probability in the label
        #label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        
        # display the label and bounding box rectangle on the output
        # frame
        if(label=="Mask"):    
            cv2.putText(frame,"Mask: You are allowed", (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        elif(label=="No Mask"):
            lab="No Mask: You are not allowed"
            cv2.putText(frame, lab, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()

vs.release()

Detection 0: confidence=0.7875673770904541
Bounding box: startX=187, startY=99, endX=298, endY=233
Face shape: (134, 111, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Detection 0: confidence=0.8878187537193298
Bounding box: startX=188, startY=94, endX=296, endY=226
Face shape: (132, 108, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Detection 0: confidence=0.8395746350288391
Bounding box: startX=186, startY=89, endX=292, endY=226
Face shape: (137, 106, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Detection 0: confidence=0.8253311514854431
Bounding box: startX=183, startY=84, endX=293, endY=228
Face shape: (144, 110, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Detection 0: confidence=0.9551381468772888
Bounding box: startX=183, startY=84, endX=292, endY=225
Face shape: (141, 109, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Detection 0: confidence=0.9756070375442505
Bounding box: startX=181, startY=79, endX=291, endY=221
Face shape: (142, 110, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Detection 0: confidenc

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Detection 0: confidence=0.5460401773452759
Bounding box: startX=201, startY=52, endX=305, endY=196
Face shape: (144, 104, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Detection 0: confidence=0.5104081630706787
Bounding box: startX=203, startY=53, endX=305, endY=194
Face shape: (141, 102, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Detection 0: confidence=0.6517587304115295
Bounding box: startX=204, startY=49, endX=305, endY=192
Face shape: (143, 101, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Detection 0: confidence=0.7252952456474304
Bounding box: startX=208, startY=52, endX=306, endY=196
Face shape: (144, 98, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Detection 0: confidence=0.6277663111686707
Bounding box: startX=207, startY=53, endX=307, endY=194
Face shape: (141, 100, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Detection 0: confidence=0.7421972155570984
Bounding box: startX=208, startY=53, endX=307, endY=195
Face shape: (142, 99, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Detection 0: confidence=0.970615804195404
Bounding box: startX=217, startY=44, endX=320, endY=193
Face shape: (149, 103, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Detection 0: confidence=0.8615137934684753
Bounding box: startX=213, startY=61, endX=311, endY=203
Face shape: (142, 98, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Detection 0: confidence=0.8689196705818176
Bounding box: startX=211, startY=72, endX=308, endY=210
Face shape: (138, 97, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Detection 0: confidence=0.746982753276825
Bounding box: startX=210, startY=69, endX=308, endY=209
Face shape: (140, 98, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Detection 0: confidence=0.925187885761261
Bounding box: startX=198, startY=74, endX=306, endY=207
Face shape: (133, 108, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Detection 0: confidence=0.843978762626648
Bounding box: startX=204, startY=76, endX=310, endY=209
Face shape: (133, 106, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Detection 0: confidence=0.9998923540115356
Bounding box: startX=191, startY=42, endX=300, endY=180
Face shape: (138, 109, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Detection 0: confidence=0.9991509914398193
Bounding box: startX=196, startY=40, endX=304, endY=179
Face shape: (139, 108, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Detection 0: confidence=0.9961723685264587
Bounding box: startX=205, startY=47, endX=311, endY=186
Face shape: (139, 106, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Detection 0: confidence=0.9992930889129639
Bounding box: startX=209, startY=47, endX=314, endY=188
Face shape: (141, 105, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Detection 0: confidence=0.9998465776443481
Bounding box: startX=213, startY=50, endX=317, endY=188
Face shape: (138, 104, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Detection 0: confidence=0.9999315738677979
Bounding box: startX=214, startY=49, endX=320, endY=188
Face shape: (139, 106, 3)
1/1 ━━━━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Detection 0: confidence=0.9986880421638489
Bounding box: startX=196, startY=56, endX=298, endY=193
Face shape: (137, 102, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Detection 0: confidence=0.9983413219451904
Bounding box: startX=197, startY=55, endX=299, endY=193
Face shape: (138, 102, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Detection 0: confidence=0.9969635605812073
Bounding box: startX=198, startY=55, endX=301, endY=194
Face shape: (139, 103, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Detection 0: confidence=0.9968781471252441
Bounding box: startX=197, startY=58, endX=301, endY=198
Face shape: (140, 104, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Detection 0: confidence=0.9958882927894592
Bounding box: startX=198, startY=58, endX=301, endY=196
Face shape: (138, 103, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Detection 0: confidence=0.9944936037063599
Bounding box: startX=199, startY=60, endX=302, endY=197
Face shape: (137, 103, 3)
1/1 ━━━━━━━━━━━━━━━━━━

AttributeError: 'WebcamVideoStream' object has no attribute 'release'